<a href="https://colab.research.google.com/github/RRustom/3d-ai-playground/blob/master/nglod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Geometric Level of Detail: Real-time Rendering with Implicit 3D Surfaces

[Project page](https://nv-tlabs.github.io/nglod/)

In [2]:
#@title Get Conda

!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:23
🔁 Restarting kernel...


In [1]:
#@title Check Conda was properly isntalled

!conda --version

conda 4.14.0


In [2]:
#@title Get nglod

!git clone https://github.com/nv-tlabs/nglod
%cd nglod

Cloning into 'nglod'...
remote: Enumerating objects: 475, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 475 (delta 0), reused 0 (delta 0), pack-reused 474
Receiving objects: 100% (475/475), 80.93 MiB | 29.91 MiB/s, done.
Resolving deltas: 100% (233/233), done.
/content/nglod


In [3]:
#@title Install dependencies

!conda create -n nglod python=3.8
!conda activate nglod
!pip install --upgrade pip
!pip install -r ./infra/requirements.txt

Solving environment: \ | / done

## Package Plan ##

  environment location: /usr/local/envs/nglod

  added / updated specs:
    - python=3.8


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.9.24  |       ha878542_0         150 KB  conda-forge
    libsqlite-3.39.3           |       h753d276_0         789 KB  conda-forge
    libuuid-2.32.1             |    h7f98852_1000          28 KB  conda-forge
    libzlib-1.2.12             |       h166bdaf_3          65 KB  conda-forge
    openssl-3.0.5              |       h166bdaf_2         2.8 MB  conda-forge
    python-3.8.13              |ha86cf86_0_cpython        25.2 MB  conda-forge
    setuptools-65.3.0          |     pyhd8ed1ab_1         782 KB  conda-forge
    sqlite-3.39.3              |       h4ff8645_0         789 KB  conda-forge
    ------------------------------------------------------------
                

In [4]:
#@title Get system libraries
!apt-get install libopenexr-dev 
!pip install pyexr

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libopenexr-dev is already the newest version (2.2.0-11.1ubuntu1.9).
libopenexr-dev set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for pyexr: filename=PyEXR-0.3.10-py2.py3-none-any.whl size=4493 sha256=c8b1a71fd3227449f6c4efa6359e84d3f1bb47671b148fe18626b3787f4115b9
  Stored in directory: /root/.cache/pip/wheels/da/c8/fb/f00811aa6cb7b74d76ab5941aea24ecf85fd8ca26f1a53ee39
  Created wheel for OpenEXR: filename=OpenEXR-1.3.9-cp37-cp37m-linux_x86_64.whl size=219262 sha256=3935cbcdca340b6e799dcf333c49d80b3b375e109c11413c5d4b3587

In [5]:
#@title Build CUDA extensions

%cd sdf-net/lib/extensions
!chmod +x build_ext.sh && ./build_ext.sh

/content/nglod/sdf-net/lib/extensions
running clean
'build/lib.linux-x86_64-cpython-37' does not exist -- can't clean it
'build/bdist.linux-x86_64' does not exist -- can't clean it
'build/scripts-3.7' does not exist -- can't clean it
running install
/usr/local/lib/python3.7/site-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,
/usr/local/lib/python3.7/site-packages/setuptools/command/easy_install.py:147: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  EasyInstallDeprecationWarning,
running bdist_egg
running egg_info
creating mesh2sdf.egg-info
writing mesh2sdf.egg-info/PKG-INFO
writing dependency_links to mesh2sdf.egg-info/dependency_links.txt
writing top-level names to mesh2sdf.egg-info/top_level.txt
writing manifest file 'mesh2sdf.egg-info/SOURCES.txt'
/usr/local/lib

In [11]:
#@title Download sample data

%cd /content/nglod/sdf-net/

!wget https://raw.githubusercontent.com/alecjacobson/common-3d-test-models/master/data/armadillo.obj -P data/
!wget https://raw.githubusercontent.com/nidorx/matcaps/master/1024/6E8C48_B8CDA7_344018_A8BC94.png -O data/matcap/green.png

/content/nglod/sdf-net
--2022-09-25 10:41:04--  https://raw.githubusercontent.com/nidorx/matcaps/master/1024/6E8C48_B8CDA7_344018_A8BC94.png
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 574396 (561K) [image/png]
Saving to: ‘data/matcap/green.png’

data/matcap/green.p 100%[===================>] 560.93K  --.-KB/s    in 0.02s   

2022-09-25 10:41:04 (24.9 MB/s) - ‘data/matcap/green.png’ saved [574396/574396]



In [12]:
#@title Train the model

!python app/main.py \
    --net OctreeSDF \
    --num-lods 5 \
    --dataset-path data/armadillo.obj \
    --epoch 250 \
    --exp-name armadillo

[25/09 10:42:22] [INFO] Parameters: 
```{ 'dataset': { 'analytic': False,
               'block_res': 7,
               'build_dataset': False,
               'dataset_path': 'data/armadillo.obj',
               'exclude': None,
               'get_normals': False,
               'glsl_path': '../sdf-viewer/data-files/sdf',
               'include': None,
               'mesh_batch': False,
               'mesh_dataset': 'MeshDataset',
               'mesh_subset_size': -1,
               'num_samples': 100000,
               'raw_obj_path': None,
               'sample_mode': ['rand', 'near', 'near', 'trace', 'trace'],
               'sample_tex': False,
               'samples_per_voxel': 256,
               'train_valid_split': None,
               'trim': False,
               'viewer_path': '../sdf-viewer'},
  'global': { 'debug': False,
              'exp_name': 'armadillo',
              'ngc': False,
              'perf': False,
              'seed': None,
              'valid_

In [ ]:
#@title Rendering the Model

!python app/sdf_renderer.py \
    --net OctreeSDF \
    --num-lods 5 \
    --pretrained _results/models/armadillo.pth \
    --render-res 1280 720 \
    --shading-mode matcap \
    --lod 4